Milestones 1

Nama : Putrima Azizi Al Fauztina

Batch: CODA-RMT-012

Milestones 1 bertujuan untuk melakukan ETL mulai dari web scraping (Extract) lalu membersihkan data hasil scraping, kemudian menyimpan hasil cleansing data ke file CSV (Transform).

### 1. Import Library

In [1]:
# Untuk olah data
import pandas as pd


# Webscraping
from selenium import webdriver
from bs4 import BeautifulSoup
import time

In [2]:
# Create driver Chrome untuk menjalankan browser otomatis
driver = webdriver.Chrome()

# List untuk menampung data
nama_produk = []
warna = []
harga_asli = []
harga_diskon = []
rating_produk = []

# Link
for i in range(1,3):
    url = f'https://thenblank.com/collections/all?page={i}'

# untuk membuka halaman (URL) tersebut di browser otomatis
    driver.get(url)

# Memberi waktu untuk halaman termuat selama 10 detik
    time.sleep(10)

# ambil html suatu website
    html = driver.page_source

# Clean Page source
    soup = BeautifulSoup(html, 'html.parser')

# Mendapatkan semua box produk
    boxes = soup.find_all('div', {'class':'product-block__inner'})

    for box in boxes:
        try:
            nama_box = box.find('span',{'class':'title'})
            nama_produk.append(nama_box.get_text())
        except:
            nama_produk.append(None)

        try:
            warna_box = box.find('span',{'class':'subtitle'})
            warna.append(warna_box.get_text())
        except:
            warna.append(None)
    
        # Ambil data harga_asli & harga_diskon
        try:
        # Cari harga
            harga_1 = box.find('span', {'class':'was-price theme-money'})
        
            if harga_1:
            # Cari harga normal
                harga_asli.append(harga_1.get_text(strip=True))
            
            # Cari harga diskon
                harga_2 = box.find('span', {'class':'theme-money'})
                harga_diskon.append(harga_2.get_text(strip=True))
            else:
            # Kalau tidak ada harga diskon = harga normal
                harga_normal = box.find('span', {'class':'theme-money'})
                harga_asli.append(harga_normal.get_text(strip=True))
                harga_diskon.append(None)
        except:
            harga_asli.append(None)
            harga_diskon.append(None)

        try:
            rating_produk_box = box.find('div', {'class':'ssw-widget-avg-rate-listing ssw-stars ssw-stars-large'})
            if  rating_produk_box and rating_produk_box.get('data-rate'):
                rating_produk.append(rating_produk_box['data-rate'])
            else:
                rating_produk.append(None)
        except:
            rating_produk.append(None)

In [3]:
# membuat dataframe untuk menyimpan list data hasil scraping
df = pd.DataFrame()
df['nama_produk'] = nama_produk
df['warna_keterangan'] = warna
df['harga_asli'] = harga_asli
df['harga_diskon'] = harga_diskon
df['rating_produk'] = rating_produk

In [4]:
# Menampilkan seluruh data
df

,nama_produk,warna_keterangan,harga_asli,harga_diskon,rating_produk
0,Additional,Gift Card,Rp 6.000,None,5
1,Ankle Chino Pants,Greymist,Rp 169.000,Rp 89.500,5
2,Ankle Chino Pants,Sand,Rp 169.000,None,5
3,Ankle Culotte,Almond,Rp 189.000,Rp 149.250,5
4,Ankle Culotte,Barri,Rp 189.000,None,5
...,...,...,...,...,...
91,Everyday Shirt,Desert,Rp 169.000,Rp 149.000,5
92,Everyday Shirt,Dusty,Rp 149.000,None,5
93,Everyday Shirt,Lavender,Rp 149.000,None,None
94,Everyday Shirt,Maroon,Rp 149.000,None,5


In [5]:
# menutup browser otomatis / WebDriver
driver.quit()

In [7]:
# cek struktur data tabel
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   nama_produk       96 non-null     object
 1   warna_keterangan  96 non-null     object
 2   harga_asli        96 non-null     object
 3   harga_diskon      13 non-null     object
 4   rating_produk     46 non-null     object
dtypes: object(5)
memory usage: 3.9+ KB


In [8]:
# Menyimpan data hasil scraping ke file CSV
df.to_csv("data_produk_raw.csv", index=False)

In [9]:
# Menampilkan data
df = pd.read_csv("data_produk_raw.csv")
df.head(10)

,nama_produk,warna_keterangan,harga_asli,harga_diskon,rating_produk
0,Additional,Gift Card,Rp 6.000,NaN,5.0
1,Ankle Chino Pants,Greymist,Rp 169.000,Rp 89.500,5.0
2,Ankle Chino Pants,Sand,Rp 169.000,NaN,5.0
3,Ankle Culotte,Almond,Rp 189.000,Rp 149.250,5.0
4,Ankle Culotte,Barri,Rp 189.000,NaN,5.0
5,Ankle Culotte,Black,Rp 189.000,NaN,5.0
6,Ankle Culotte,Broken White,Rp 189.000,NaN,5.0
7,Ankle Culotte,Buckwheat,Rp 189.000,NaN,5.0
8,Ankle Culotte,Khaki,Rp 189.000,NaN,5.0
9,Ankle Culotte,Nude,Rp 189.000,NaN,NaN


In [10]:
# Cek missing values
print(df.isna().sum())

nama_produk          0
warna_keterangan     0
harga_asli           0
harga_diskon        83
rating_produk       50
dtype: int64


In [11]:
# Data kosong/missing value kolom harga_diskon, diisi dengan harga_asli (tidak ada diskon)
df['harga_diskon'] = df['harga_diskon'].fillna(df['harga_asli'])

In [12]:
# ubah tipe data kolom rating_produk menjadi float
df['rating_produk'] = pd.to_numeric(df['rating_produk'], errors='coerce')

In [13]:
# Hapus spasi di awal dan akhir teks untuk kolom nama_produk dan warna_keterangan
df['nama_produk'] = df['nama_produk'].str.strip()
df['warna_keterangan'] = df['warna_keterangan'].str.strip()

In [14]:
# Cek missing values
print(df.isna().sum())

nama_produk          0
warna_keterangan     0
harga_asli           0
harga_diskon         0
rating_produk       50
dtype: int64


notes: missing value pada kolom rating_produk dibiarkan tetap kosong/NaN agar tidak menggubah ringkasan nilai statistik df.describe()

In [15]:
# cleansing data harga_asli dan ubah tipe data menjadi integer
df['harga_asli'] = df['harga_asli'].astype(str)        # ubah ke string
df['harga_asli'] = df['harga_asli'].str.replace('Rp', '', regex=False) # Menghapus teks 'Rp' dari kolom harga
df['harga_asli'] = df['harga_asli'].str.replace('.', '', regex=False) # Menghapus tanda titik (.) dari kolom harga
df['harga_asli'] = df['harga_asli'].str.replace(' ', '', regex=False) # Menghapus spasi di dalam kolom harga
df['harga_asli'] = df['harga_asli'].astype(int) # ubah jadi angka (integer)

# cleansing data harga_diskon dan ubah tipe data menjadi integer
df['harga_diskon'] = df['harga_diskon'].astype(str) # ubah ke string
df['harga_diskon'] = df['harga_diskon'].str.replace('Rp', '', regex=False) # Menghapus teks 'Rp' dari kolom harga
df['harga_diskon'] = df['harga_diskon'].str.replace('.', '', regex=False) # Menghapus tanda titik (.) dari kolom harga
df['harga_diskon'] = df['harga_diskon'].str.replace(' ', '', regex=False) # Menghapus spasi di dalam kolom harga
df['harga_diskon'] = df['harga_diskon'].astype(int) # ubah jadi angka (integer)

In [16]:
# cek struktur data tabel setelah cleansing data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   nama_produk       96 non-null     object 
 1   warna_keterangan  96 non-null     object 
 2   harga_asli        96 non-null     int64  
 3   harga_diskon      96 non-null     int64  
 4   rating_produk     46 non-null     float64
dtypes: float64(1), int64(2), object(2)
memory usage: 3.9+ KB


In [17]:
# cek statistik dari data angka
df.describe()

,harga_asli,harga_diskon,rating_produk
count,96.000000,96.000000,46.000000
mean,200843.750000,191403.645833,4.984783
std,70189.417125,71290.808902,0.046986
min,6000.000000,6000.000000,4.800000
25%,149000.000000,147750.000000,5.000000
50%,189000.000000,189000.000000,5.000000
75%,244000.000000,239000.000000,5.000000
max,359000.000000,339000.000000,5.000000


In [18]:
# menampilkan hasil cleansing data
df

,nama_produk,warna_keterangan,harga_asli,harga_diskon,rating_produk
0,Additional,Gift Card,6000,6000,5.0
1,Ankle Chino Pants,Greymist,169000,89500,5.0
2,Ankle Chino Pants,Sand,169000,169000,5.0
3,Ankle Culotte,Almond,189000,149250,5.0
4,Ankle Culotte,Barri,189000,189000,5.0
...,...,...,...,...,...
91,Everyday Shirt,Desert,169000,149000,5.0
92,Everyday Shirt,Dusty,149000,149000,5.0
93,Everyday Shirt,Lavender,149000,149000,NaN
94,Everyday Shirt,Maroon,149000,149000,5.0


In [19]:
# Menyimpan hasil cleansing data ke file CSV
df.to_csv("data_produk_clean.csv", index=False)